In [1]:
# Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.sql.types import *
%matplotlib inline

In [2]:
spark = SparkSession.builder.master("local").appName('Ops').getOrCreate()

In [3]:
path = "E:/Rutgers/Projects/MDSR/IPL-MSDR"

In [ ]:
# Reading Data
matches = pd.read_csv(path + '/dataset/original_ipldata/matches.csv')
deliveries = pd.read_csv(path + '/dataset/original_ipldata/deliveries.csv')

In [ ]:
# Schema of original data (matches.csv)
matches.info()

In [ ]:
# Schema of original data (deliveries.csv)
deliveries.info()

# Data Cleaning

In [ ]:
# Dropping columns that are of no use
matches = matches.drop(columns = ['umpire1', 'umpire2','umpire3','date'])

In [ ]:
#Filing missing values
matches['winner'].fillna('Draw', inplace=True)
matches['city'].fillna('Dubai',inplace=True)
deliveries = deliveries.fillna(value = 0)

In [ ]:
# Fixing error Data
matches = matches.replace('Rising Pune Supergiants', 'Rising Pune Supergiant')

In [ ]:
# Schema of cleaned data (matches.csv)
matches.info()

In [ ]:
# Schema of cleaned data (deliveries.csv)
deliveries.info()

In [ ]:
# Saving cleaned data (matches.csv)
matches.to_csv(path + '/dataset/clean_data/matches.csv')

In [ ]:
# Saving cleaned data (deliveries.csv)
deliveries.to_csv(path + '/dataset/clean_data/deliveries.csv')

# Basic Analysis

In [ ]:
# Teams playing in the league
teams = matches['team1'].unique()
print("Total number of teams participated so far: " + str(len(matches['team1'].unique())))
print("Teams participated so far: ")
for i in teams:
    print("- " + i)

In [ ]:
# Total Venues
print("Number of venues matches were played: " + str(len(matches['venue'].unique())))
for i in matches['venue'].unique():
    print("- " + i)

In [ ]:
# Cities the matches were played
print("Number of cities matches were played: " + str(len(matches['city'].unique())))
for i in matches['city'].unique():
    print("- " + i)

In [ ]:
# Total number of bowlers so far
print("Total number of bowlers: " + str(len(deliveries['bowler'].unique())))

In [ ]:
# Total number of batsmen so far
print("Total number of batsmen: " + str(len(deliveries['batsman'].unique())))

In [ ]:
# Total number of participating players
players = set()
for i in range(len(deliveries['match_id'])):
    players.add(deliveries['bowler'][i])
    players.add(deliveries['batsman'][i])
    players.add(deliveries['non_striker'][i])
print("Total number of player: " + str(len(players)))

# Spark Analysis

In [ ]:
# Reading Data
matches = spark.read.csv(path + '/dataset/clean_data/matches.csv',inferSchema=True,header=True)
deliveries = spark.read.csv(path + '/dataset/clean_data/deliveries.csv',inferSchema=True,header=True)

### Total number of matches per season

In [ ]:
matches.registerTempTable('seasons')
seasons = spark.sql('''Select distinct(season),count(*) as total_matches from seasons group by season ''') 
seasons.show()

In [ ]:
# Plot
fig, a = plt.subplots()
a = sns.barplot(x ="season", y="total_matches", data=seasons.toPandas(),palette='viridis')
a.set_xlabel('Season')
a.set_ylabel('Total Matches')
a.set_title('Number of matches in each season')

### Number of maches played by each team since season 1

In [ ]:
matches.registerTempTable('team')
team = spark.sql('''Select distinct(team), count(*) as total_matches from (Select team1 as team from team UNION ALL (select team2 as team from team)) group by team ''')
team.show()

In [ ]:
# Plot
fig, a = plt.subplots(figsize = (5,5))
a = sns.barplot(x ="total_matches", y="team", data=team.toPandas(), palette='viridis')
a.set_ylabel('Team')
a.set_xlabel('Total Matches')
a.set_title('Number of matches played by each team')

### Total season in which teams have played

In [ ]:
matches.registerTempTable('team_season')
team_season = spark.sql('''Select team1 as team, min(season) as first_season, max(season) as last_season, count(distinct(season)) as total_seasons from team_season group by team1 order by total_seasons desc''')
team_season.show()

### Total number of matches won by teams

In [ ]:
matches.registerTempTable('most_win')
most_win = spark.sql('''Select distinct(winner) as team, count(*) as total_matches from most_win where winner <>'None' group by winner order by total_matches ''')
most_win.show()

In [ ]:
# Plot
fig, a = plt.subplots(figsize = (5,5))
a = sns.barplot(x ="total_matches", y="team", data=most_win.toPandas(), palette='viridis')
a.set_ylabel('Team')
a.set_xlabel('Total Matches')
a.set_title('Number of matches won by each team')

### Total matches won by teams in each season

In [ ]:
matches.registerTempTable('most_win_by_season')
most_win_by_season = spark.sql('''Select season, winner as team, count(*) as total_matches_won from most_win_by_season where winner <> 'None' group by season, winner order by total_matches_won desc''')
most_win_by_season.show()

### Players with maximum man of the match awards 

In [ ]:
matches.registerTempTable('man_match')
man_match = spark.sql('''Select distinct(player_of_match), count(*) as total_matches from man_match group by player_of_match order by total_matches desc limit 10 ''')
man_match.show()

In [ ]:
# Plot
fig, a = plt.subplots(figsize = (5,5))
a = sns.barplot(x ="total_matches", y="player_of_match", data=man_match.toPandas(), palette='viridis')
a.set_xlabel('Total Matches')
a.set_ylabel('Player')
a.set_title('Number of times player won man of the match')

### Number of matches per Venue

In [ ]:
matches.registerTempTable('venue')
venue = spark.sql('''Select distinct(venue), count(*) as total_matches from venue group by venue''')
venue.show()

In [ ]:
# Plot
fig, a = plt.subplots(figsize = (10,20))
a = sns.barplot(x ="total_matches", y="venue", data=venue.toPandas(), palette='viridis')
a.set_ylabel('Venue')
a.set_xlabel('Total Matches')
a.set_title('Number of matches at each venue')

### Percentage toss decisions 

In [ ]:
matches.registerTempTable('toss')
toss = spark.sql('''Select distinct(toss_decision), ((count(toss_decision)*100)/ (select count(*) from toss)) as percentage_count from toss group by toss_decision''')
toss.show()

In [ ]:
# Plot
fig, a = plt.subplots(figsize = (5,5))
a = sns.barplot(x ="toss_decision", y="percentage_count", data=toss.toPandas(), palette='viridis')
a.set_ylabel('Percentage')
a.set_xlabel('Toss Decision')
a.set_title('Percentage Plot of toss_decision')

### Percentage of team winning the toss as well as the match

In [ ]:
matches.registerTempTable('toss_and_won')
matches.registerTempTable('toss_won_data')
toss_won_data = spark.sql('''Select t1.season, t1.total_matches, \
          t2.count_toss_and_won as count_toss_and_won, \
          (t2.count_toss_and_won / t1.total_matches * 100) as percent_toss_and_won from \
          (Select distinct(season),count(*) as total_matches from seasons group by season)t1 \
          left join (Select distinct(season), count(*) as count_toss_and_won from toss_and_won where toss_winner = winner group by season)t2 on t1.season = t2.season order by season''')
toss_won_data.show()

In [ ]:
# Plot
fig, a = plt.subplots(figsize = (10,5))
a = sns.barplot(x ="season", y="percent_toss_and_won", data=toss_won_data.toPandas(), palette='viridis')
a.set_ylabel('Percentage')
a.set_xlabel('Season')
a.set_title('Percentage Plot of Season and Toss_and_won')

### Percentage matches won by batting first 

In [ ]:
win_batting_first = spark.sql('''Select t1.season, t1.total_matches, \
          t2.win_batting_first as win_batting_first, \
          (t2.win_batting_first/ t1.total_matches * 100) as percent_win_batting_first from \
          (Select distinct(season),count(*) as total_matches from seasons group by season)t1 \
          left join (Select distinct(season), count(*) as win_batting_first from seasons where win_by_runs > 0  group by season)t2 on t1.season = t2.season order by season ''')
win_batting_first.show()

In [ ]:
# Plot
fig, a = plt.subplots(figsize = (10,5))
a = sns.barplot(x ="season", y="percent_win_batting_first", data=win_batting_first.toPandas(), palette='viridis')
a.set_ylabel('Percentage')
a.set_xlabel('Season')
a.set_title('Percentage Plot of Season and won by batting')

### Percentage matches won by fielding first

In [ ]:
win_bowling_first = spark.sql('''Select t1.season, t1.total_matches, \
          t2.win_bowling_first as win_bowling_first, \
          (t2.win_bowling_first/ t1.total_matches * 100) as percent_win_bowling_first from \
          (Select distinct(season),count(*) as total_matches from seasons group by season)t1 \
          left join (Select distinct(season), count(*) as win_bowling_first from seasons where win_by_wickets > 0  group by season)t2 on t1.season = t2.season order by season ''')
win_bowling_first.show()

In [ ]:
# Plot
fig, a = plt.subplots(figsize = (10,5))
a = sns.barplot(x ="season", y="percent_win_bowling_first", data=win_bowling_first.toPandas(), palette='viridis')
a.set_ylabel('Percentage')
a.set_xlabel('Season')
a.set_title('Percentage Plot of Season and won by wickets ')

# Final Analysis

In [4]:
# Reading data
matches = spark.read.csv(path + '/dataset/clean_data/matches.csv',inferSchema=True,header=True)
deliveries = spark.read.csv(path + '/dataset/clean_data/deliveries.csv',inferSchema=True,header=True)

In [5]:
# Creating temporary tables of the data
matches.registerTempTable('matches_db')
deliveries.registerTempTable('deliveries_db')

In [6]:
# Merging both the tables
merged_db = spark.sql('select m.*,d.* from matches_db as m inner join deliveries_db as d on m.id=d.match_id')
merged_db.registerTempTable('analysis_db')

## Batting Metrics 

In [7]:
# nmba: no. of batsmen
# nm: no. of matches played by a batsman
# hha: hard hitting ability
# f: finisher
# fsa: fast scoring ability
# con: consistency
# rbw: running between wickets

In [8]:
# Number of Batsmen
nmba = spark.sql('select count(distinct(batsman)) as No_of_Batsman from analysis_db')
nmba.show()

+-------------+
|No_of_Batsman|
+-------------+
|          516|
+-------------+



In [9]:
# Number of Matches played by a batsmen
nm = spark.sql('select batsman, count(distinct(match_id)) as No_of_Matches \
                from analysis_db group by batsman')
nm.registerTempTable('no_of_matches_table')

### Hard Hitting Ability 

In [10]:
# Hard Hitting Ability = (4*Fours + 6*Sixes)/Balls Played by Batsman
hha = spark.sql('select nmt.batsman as Batsman, round(nvl(t4.hard_hitting_ability,0), 5) as \
                Hard_Hitting_Ability from \
                (select t1.batsman, (t1.fours*4 + t2.sixes*6)/t3.balls_played as hard_hitting_ability\
                from (select batsman,count(*) as fours from analysis_db where batsman_runs = 4 group by batsman) t1 \
                inner join  \
                (select batsman,count(*) as sixes from analysis_db where batsman_runs = 6 \
                group by batsman) t2 on t1.batsman=t2.batsman\
                inner join\
                (select batsman,count(*) as balls_played from analysis_db \
                group by batsman) t3 on t3.batsman=t1.batsman) t4 \
                right join \
                no_of_matches_table nmt on t4.batsman = nmt.batsman')
hha.registerTempTable('hard_hitting_ability')

In [11]:
hha = spark.sql('select rank() over (order by Hard_Hitting_Ability desc) as Rank, t1.* \
                  from hard_hitting_ability t1 \
                  inner join \
                  no_of_matches_table t2\
                  on t1.batsman = t2.batsman where no_of_matches>9')
hha.registerTempTable('hard_hitting_ability')
hha.show(10)

+----+-------------+--------------------+
|Rank|      Batsman|Hard_Hitting_Ability|
+----+-------------+--------------------+
|   1|   AD Russell|             1.37733|
|   2|    SP Narine|             1.33056|
|   3|        M Ali|             1.21311|
|   4|    KK Cooper|                 1.2|
|   5|  BCJ Cutting|             1.19178|
|   6|    K Gowtham|             1.16279|
|   7|CR Brathwaite|             1.13333|
|   8|     CH Gayle|             1.10699|
|   9|  Rashid Khan|             1.10448|
|  10|   GJ Maxwell|             1.09313|
+----+-------------+--------------------+
only showing top 10 rows



In [12]:
hha = spark.sql('select t1.*, round((240-rank)/240, 5) as Points, round((240-rank)*1.5/240, 5) as Weights \
                from hard_hitting_ability t1')
hha.registerTempTable('hard_hitting_ability')
hha.show(10)

+----+-------------+--------------------+-------+-------+
|Rank|      Batsman|Hard_Hitting_Ability| Points|Weights|
+----+-------------+--------------------+-------+-------+
|   1|   AD Russell|             1.37733|0.99583|1.49375|
|   2|    SP Narine|             1.33056|0.99167|1.48750|
|   3|        M Ali|             1.21311| 0.9875|1.48125|
|   4|    KK Cooper|                 1.2|0.98333|1.47500|
|   5|  BCJ Cutting|             1.19178|0.97917|1.46875|
|   6|    K Gowtham|             1.16279|  0.975|1.46250|
|   7|CR Brathwaite|             1.13333|0.97083|1.45625|
|   8|     CH Gayle|             1.10699|0.96667|1.45000|
|   9|  Rashid Khan|             1.10448| 0.9625|1.44375|
|  10|   GJ Maxwell|             1.09313|0.95833|1.43750|
+----+-------------+--------------------+-------+-------+
only showing top 10 rows



### Finisher 

In [13]:
# Finisher = Not Out innings/Total Innings played
f = spark.sql('select t3.batsman as Batsman, round(t3.not_out_innings/t4.total_matches_played, 5) as Finisher from\
              (select t1.batsman, t1.matches_played-t2.number_of_times_out as not_out_innings from \
              (select batsman, count(distinct(match_id)) as matches_played from analysis_db group by batsman) t1\
              inner join \
              (select batsman, count(*) as number_of_times_out from analysis_db where player_dismissed = batsman group by batsman) t2\
              on t1.batsman=t2.batsman) t3\
              inner join\
              (select batsman, count(distinct(match_id)) as total_matches_played \
              from analysis_db group by batsman) t4\
              on t3.batsman = t4.batsman')
f.registerTempTable('finisher')

In [14]:
f = spark.sql('select rank() over (order by finisher desc) as Rank, t1.* \
              from finisher t1 \
              inner join \
              no_of_matches_table t2 \
              on t1.batsman = t2.batsman \
              where no_of_matches>9')
f.registerTempTable('finisher')
f.show(10)

+----+--------------+--------+
|Rank|       Batsman|Finisher|
+----+--------------+--------+
|   1| Iqbal Abdulla| 0.92308|
|   2|      A Kumble| 0.86667|
|   3|Sandeep Sharma| 0.78571|
|   4|   S Sreesanth|    0.75|
|   5|     S Aravind|     0.7|
|   5|     JJ Bumrah|     0.7|
|   5|      VR Aaron|     0.7|
|   8|     YS Chahal| 0.66667|
|   8|      I Sharma| 0.66667|
|  10|  Bipul Sharma| 0.64706|
+----+--------------+--------+
only showing top 10 rows



In [15]:
f = spark.sql('select t1.*, round((240-rank)/240, 5) as Points, round((240-rank)*1.5/240, 5) as Weights \
              from finisher t1')
f.registerTempTable('finisher')
f.show(10)

+----+--------------+--------+-------+-------+
|Rank|       Batsman|Finisher| Points|Weights|
+----+--------------+--------+-------+-------+
|   1| Iqbal Abdulla| 0.92308|0.99583|1.49375|
|   2|      A Kumble| 0.86667|0.99167|1.48750|
|   3|Sandeep Sharma| 0.78571| 0.9875|1.48125|
|   4|   S Sreesanth|    0.75|0.98333|1.47500|
|   5|     S Aravind|     0.7|0.97917|1.46875|
|   5|     JJ Bumrah|     0.7|0.97917|1.46875|
|   5|      VR Aaron|     0.7|0.97917|1.46875|
|   8|     YS Chahal| 0.66667|0.96667|1.45000|
|   8|      I Sharma| 0.66667|0.96667|1.45000|
|  10|  Bipul Sharma| 0.64706|0.95833|1.43750|
+----+--------------+--------+-------+-------+
only showing top 10 rows



### Fast Scoring Ability

In [16]:
# Fast Scoring Ability = Total Runs/Balls Played by Batsman
fsa = spark.sql('select batsman as Batsman, round(Total_Runs/balls_played, 5) as Fast_Scoring_Ability \
                  from (select batsman,sum(batsman_runs) as Total_Runs, count(*) as balls_played \
                  from analysis_db group by batsman)')
fsa.registerTempTable('fast_scoring_ability')

In [17]:
fsa = spark.sql('select rank() over (order by fast_scoring_ability desc) as Rank, t1.* \
                  from fast_scoring_ability t1 \
                  inner join \
                  no_of_matches_table t2 \
                  on t1.batsman = t2.batsman where no_of_matches>9')
fsa.registerTempTable('fast_scoring_ability')
fsa.show(10)

+----+-------------+--------------------+
|Rank|      Batsman|Fast_Scoring_Ability|
+----+-------------+--------------------+
|   1|   AD Russell|              1.7995|
|   2|    K Gowtham|             1.72093|
|   3|        M Ali|             1.69945|
|   4|    SP Narine|             1.66944|
|   5|    KK Cooper|             1.65714|
|   6|  BCJ Cutting|             1.64384|
|   7|  Rashid Khan|             1.62687|
|   8|      RR Pant|             1.62319|
|   9|   J Bairstow|             1.59727|
|  10|CR Brathwaite|             1.56667|
+----+-------------+--------------------+
only showing top 10 rows



In [18]:
fsa = spark.sql('select t1.*, round((240-rank)/240, 5) as Points, round((240-rank)*1.5/240, 5) as Weights \
                from fast_scoring_ability t1')
fsa.registerTempTable('fast_scoring_ability')
fsa.show(10)

+----+-------------+--------------------+-------+-------+
|Rank|      Batsman|Fast_Scoring_Ability| Points|Weights|
+----+-------------+--------------------+-------+-------+
|   1|   AD Russell|              1.7995|0.99583|1.49375|
|   2|    K Gowtham|             1.72093|0.99167|1.48750|
|   3|        M Ali|             1.69945| 0.9875|1.48125|
|   4|    SP Narine|             1.66944|0.98333|1.47500|
|   5|    KK Cooper|             1.65714|0.97917|1.46875|
|   6|  BCJ Cutting|             1.64384|  0.975|1.46250|
|   7|  Rashid Khan|             1.62687|0.97083|1.45625|
|   8|      RR Pant|             1.62319|0.96667|1.45000|
|   9|   J Bairstow|             1.59727| 0.9625|1.44375|
|  10|CR Brathwaite|             1.56667|0.95833|1.43750|
+----+-------------+--------------------+-------+-------+
only showing top 10 rows



### Consistency

In [19]:
# Consistency = Total Runs/Number of Times Out
con = spark.sql('select t1.batsman as Batsman, round(t1.Total_runs/t2.no_of_times_dismissed, 5) as Consistency \
                from (select batsman,sum(batsman_runs) as Total_runs \
                from analysis_db group by batsman) t1 \
                inner join \
                (select batsman, count(*) as no_of_times_dismissed \
                from analysis_db where player_dismissed is not null \
                group by batsman) t2 on t1.batsman=t2.batsman')
con.registerTempTable('consistency')

In [20]:
con = spark.sql('select rank() over (order by consistency desc) as Rank, t1.* \
                  from consistency t1 \
                  inner join \
                  no_of_matches_table t2 \
                  on t1.batsman = t2.batsman where no_of_matches>9')
con.registerTempTable('consistency')
con.show(10)

+----+-------------+-----------+
|Rank|      Batsman|Consistency|
+----+-------------+-----------+
|   1|   AD Russell|     1.7995|
|   2|    K Gowtham|    1.72093|
|   3|        M Ali|    1.69945|
|   4|    SP Narine|    1.66944|
|   5|    KK Cooper|    1.65714|
|   6|  BCJ Cutting|    1.64384|
|   7|  Rashid Khan|    1.62687|
|   8|      RR Pant|    1.62319|
|   9|   J Bairstow|    1.59727|
|  10|CR Brathwaite|    1.56667|
+----+-------------+-----------+
only showing top 10 rows



In [21]:
con = spark.sql('select t1.*, round((240-rank)/240, 5) as Points, round((240-rank)*1.25/240, 5) as Weights \
                from consistency t1')
con.registerTempTable('consistency')
con.show(10)

+----+-------------+-----------+-------+-------+
|Rank|      Batsman|Consistency| Points|Weights|
+----+-------------+-----------+-------+-------+
|   1|   AD Russell|     1.7995|0.99583|1.24479|
|   2|    K Gowtham|    1.72093|0.99167|1.23958|
|   3|        M Ali|    1.69945| 0.9875|1.23438|
|   4|    SP Narine|    1.66944|0.98333|1.22917|
|   5|    KK Cooper|    1.65714|0.97917|1.22396|
|   6|  BCJ Cutting|    1.64384|  0.975|1.21875|
|   7|  Rashid Khan|    1.62687|0.97083|1.21354|
|   8|      RR Pant|    1.62319|0.96667|1.20833|
|   9|   J Bairstow|    1.59727| 0.9625|1.20313|
|  10|CR Brathwaite|    1.56667|0.95833|1.19792|
+----+-------------+-----------+-------+-------+
only showing top 10 rows



###  Running Between Wickets

In [22]:
# Running Between Wickets = (Total Runs – (4*Fours + 6*Sixes))/(Total Balls Played – Boundary Balls)
rbw = spark.sql('select t9.batsman as Batsman, round(nvl(t8.running_between_wickets,0), 5) as Running_Between_Wickets \
                from (select t4.batsman, t4.first_bracket/t7.second_bracket as Running_Between_Wickets \
                from (select t1.batsman, t3.total_runs-(t1.fours*4 + t2.sixes*6) as first_bracket \
                from (select batsman,count(*) as fours from analysis_db where batsman_runs = 4 \
                group by batsman) t1 \
                inner join \
                (select batsman,count(*) as sixes from analysis_db where batsman_runs = 6 group by batsman) t2 \
                on t1.batsman=t2.batsman \
                inner join \
                (select batsman,sum(batsman_runs) as total_runs from analysis_db group by batsman) t3 \
                on t3.batsman=t1.batsman) t4 \
                inner join\
                (select t5.batsman, t5.total_balls_played-t6.boundry_balls as second_bracket from \
                (select batsman, count(*) as total_balls_played from analysis_db group by batsman) t5 \
                inner join \
                (select batsman, count(*) as boundry_balls from analysis_db where batsman_runs=4 or batsman_runs=6 group by batsman) t6\
                on t5.batsman=t6.batsman) t7 \
                on t4.batsman=t7.batsman) t8 \
                right join \
                no_of_matches_table t9 \
                on t8.batsman = t9.batsman')
rbw.registerTempTable('running_between_wickets')

In [23]:
rbw = spark.sql('select rank() over (order by running_between_wickets desc) as Rank, t1.* \
                  from running_between_wickets t1 \
                  inner join \
                  no_of_matches_table t2\
                  on t1.batsman = t2.batsman where no_of_matches>9')
rbw.registerTempTable('running_between_wickets')
rbw.show(10)

+----+--------------+-----------------------+
|Rank|       Batsman|Running_Between_Wickets|
+----+--------------+-----------------------+
|   1|  Bipul Sharma|                0.85577|
|   2|       TM Head|                0.83206|
|   3|    WPUJC Vaas|                0.80882|
|   4|     V Shankar|                0.79245|
|   5|      M Kartik|                0.78218|
|   6| Mohammad Nabi|                0.77922|
|   7|     BA Stokes|                0.76471|
|   8|A Ashish Reddy|                0.76364|
|   8|     CH Morris|                0.76364|
|  10|        S Gill|                0.76235|
+----+--------------+-----------------------+
only showing top 10 rows



In [24]:
rbw = spark.sql('select t1.*, round((240-rank)/240, 5) as Points, round((240-rank)*1.25/240, 5) as Weights \
                from running_between_wickets t1')
rbw.registerTempTable('running_between_wickets')
rbw.show(10)

+----+--------------+-----------------------+-------+-------+
|Rank|       Batsman|Running_Between_Wickets| Points|Weights|
+----+--------------+-----------------------+-------+-------+
|   1|  Bipul Sharma|                0.85577|0.99583|1.24479|
|   2|       TM Head|                0.83206|0.99167|1.23958|
|   3|    WPUJC Vaas|                0.80882| 0.9875|1.23438|
|   4|     V Shankar|                0.79245|0.98333|1.22917|
|   5|      M Kartik|                0.78218|0.97917|1.22396|
|   6| Mohammad Nabi|                0.77922|  0.975|1.21875|
|   7|     BA Stokes|                0.76471|0.97083|1.21354|
|   8|A Ashish Reddy|                0.76364|0.96667|1.20833|
|   8|     CH Morris|                0.76364|0.96667|1.20833|
|  10|        S Gill|                0.76235|0.95833|1.19792|
+----+--------------+-----------------------+-------+-------+
only showing top 10 rows



In [25]:
# Table Name for each Metric
# Hard Hitting Ability: hard_hitting_ability
# Finisher: finisher
# Fast Scoring Ability: fast_scoring_ability
# Consistency: consistency
# Running Between Wickets: running_between_wickets

##  Total Batting Weights

In [26]:
total_batting_weight = spark.sql('select hht.Batsman, round((hht.Weights+f.Weights+fsa.Weights+c.Weights+rbw.Weights), 5) as Total_Batting_Weights \
                                 from hard_hitting_ability hht \
                                 inner join finisher f \
                                 on hht.Batsman = f.Batsman \
                                 inner join fast_scoring_ability fsa \
                                 on hht.Batsman = fsa.Batsman \
                                 inner join consistency c \
                                 on hht.Batsman = c.Batsman \
                                 inner join running_between_wickets rbw \
                                 on hht.Batsman = rbw.Batsman \
                                 order by Total_Batting_Weights desc')
total_batting_weight.registerTempTable('total_batting_weight')
total_batting_weight.show(100)

+-----------------+---------------------+
|          Batsman|Total_Batting_Weights|
+-----------------+---------------------+
|        CH Morris|              6.42812|
|     Bipul Sharma|              6.35417|
|        K Gowtham|              6.13646|
|Washington Sundar|              6.11771|
|        HH Pandya|              6.09062|
|         HV Patel|              5.94792|
|      BCJ Cutting|              5.85938|
|     Ankit Sharma|              5.85521|
|      Rashid Khan|              5.80000|
|            M Ali|              5.75313|
|   AB de Villiers|              5.72500|
|          SN Khan|              5.72292|
|   A Ashish Reddy|              5.72083|
|    Mohammad Nabi|              5.71250|
|       J Bairstow|              5.69376|
|        KH Pandya|              5.66563|
|         M Morkel|              5.66458|
|        JA Morkel|              5.59584|
|      MF Maharoof|              5.58854|
|          RR Pant|              5.49479|
|         MS Dhoni|              5

In [27]:
# Dropping intermediate tables
table_names = ['no_of_matches_table', 'hard_hitting_ability', 'finisher', 'fast_scoring_ability', 'consistency', 'running_between_wickets']
for table in table_names:
    cmd = 'drop table if exists {}'.format(table)
    drop = spark.sql(cmd)
check = spark.sql('show tables')
check.show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
|        |         analysis_db|       true|
|        |       deliveries_db|       true|
|        |          matches_db|       true|
|        |total_batting_weight|       true|
+--------+--------------------+-----------+



## Bowling Metrics 

In [28]:
# nmbo: no. of bowlers
# nmb: no. of matches played by a bowler
# eco: economy
# wta: wicket taking ability
# cons: consistency
# cwta: crucial wicket taking ability
# spi: short performance index

In [29]:
# Number of Bowlers
nmbo = spark.sql('Select count(distinct(bowler)) as No_of_Bowlers from analysis_db')
nmbo.show()

+-------------+
|No_of_Bowlers|
+-------------+
|          405|
+-------------+



In [30]:
# Number of matches played by a bowler
nmb = spark.sql('select bowler as Bowler, count(distinct(match_id)) as No_of_Matches from analysis_db group by bowler')
nmb.registerTempTable('no_of_matches_bowlers')

### Economy 

In [31]:
# Economy = Runs Scored/(Number of balls bowled by bowler/6)
eco = spark.sql('Select bowler as Bowler, round(runs/overs, 5) as Economy \
                from (Select bowler,round(count(*)/6) \
                as overs,sum(total_runs) as runs \
                from analysis_db \
                group by bowler)')
eco.registerTempTable('economy')

In [32]:
eco = spark.sql('select row_number() over (order by e.Economy asc) as Rank, e.*,n.No_of_Matches \
                from economy e \
                inner join \
                no_of_matches_bowlers n \
                on e.Bowler = n.Bowler where n.No_of_Matches>9')
eco.registerTempTable('economy')
eco.show(10)

+----+----------------+-------+-------------+
|Rank|          Bowler|Economy|No_of_Matches|
+----+----------------+-------+-------------+
|   1|   Sohail Tanvir|   6.25|           11|
|   2|      A Chandila|6.28205|           12|
|   3|         J Yadav|6.52632|           12|
|   4|      SM Pollock|6.53191|           13|
|   5|        A Kumble|6.64024|           42|
|   6|      GD McGrath|6.65455|           14|
|   7|        DW Steyn|6.66848|           92|
|   8|  M Muralitharan|6.68561|           66|
|   9|RN ten Doeschate|6.71429|           10|
|  10|       RD Chahar|6.71429|           15|
+----+----------------+-------+-------------+
only showing top 10 rows



In [33]:
eco = spark.sql('select *, round((212 - Rank)/212, 5) as Points, round((212 - Rank)*1.5/212, 5) as Weight from economy')
eco.registerTempTable('economy')
eco.show(10)

+----+----------------+-------+-------------+-------+-------+
|Rank|          Bowler|Economy|No_of_Matches| Points| Weight|
+----+----------------+-------+-------------+-------+-------+
|   1|   Sohail Tanvir|   6.25|           11|0.99528|1.49293|
|   2|      A Chandila|6.28205|           12|0.99057|1.48585|
|   3|         J Yadav|6.52632|           12|0.98585|1.47877|
|   4|      SM Pollock|6.53191|           13|0.98113|1.47170|
|   5|        A Kumble|6.64024|           42|0.97642|1.46462|
|   6|      GD McGrath|6.65455|           14| 0.9717|1.45755|
|   7|        DW Steyn|6.66848|           92|0.96698|1.45047|
|   8|  M Muralitharan|6.68561|           66|0.96226|1.44340|
|   9|RN ten Doeschate|6.71429|           10|0.95755|1.43632|
|  10|       RD Chahar|6.71429|           15|0.95283|1.42925|
+----+----------------+-------+-------------+-------+-------+
only showing top 10 rows



### Wicket Taking Ability

In [34]:
# Wicket Taking Ability = Number of balls bowled/Wickets Taken
wta = spark.sql('(Select t1.bowler as Bowler, round(t2.balls/t1.wickets, 5) as Wicket_Taking_Ability from \
                (Select bowler,count(*) as wickets from analysis_db where player_dismissed is not null \
                and (dismissal_kind = \'bowled\' or  dismissal_kind = \'hit wicket\' \
                or  dismissal_kind = \'stumped\' or  dismissal_kind = \'lbw\' \
                or  dismissal_kind = \'caught and bowled\' or  dismissal_kind = \'caught\') \
                group by bowler) t1 \
                inner join \
                (select count(*) as balls,bowler from analysis_db group by bowler)t2 on \
                t1.bowler = t2.bowler)')
wta.registerTempTable('wicket_taking_ability')
wta.show(10)

+--------------+---------------------+
|        Bowler|Wicket_Taking_Ability|
+--------------+---------------------+
|         A Roy|                 15.0|
| Kuldeep Yadav|             21.48718|
|    TM Dilshan|                 55.0|
|       J Botha|                28.36|
|    KA Pollard|             22.78571|
| LA Carseldine|                  7.0|
|M Muralitharan|             24.70313|
|      DR Smith|             21.42308|
|Jaskaran Singh|                 18.5|
|    A Flintoff|                 33.0|
+--------------+---------------------+
only showing top 10 rows



In [35]:
wta = spark.sql('select row_number() over (order by w.Wicket_Taking_Ability asc) as Rank, w.*, n.No_of_Matches \
                from wicket_taking_ability w \
                inner join \
                no_of_matches_bowlers n on \
                w.Bowler = n.Bowler where n.No_of_Matches>9')
wta.registerTempTable('wicket_taking_ability')
wta.show(10)

+----+--------------+---------------------+-------------+
|Rank|        Bowler|Wicket_Taking_Ability|No_of_Matches|
+----+--------------+---------------------+-------------+
|   1|       A Zampa|             11.84211|           11|
|   2| Sohail Tanvir|             12.04545|           11|
|   3|       K Ahmed|             12.68421|           10|
|   4|        N Rana|             13.42857|           12|
|   5|      K Rabada|                 14.0|           18|
|   6|      BJ Hodge|                 14.0|           20|
|   7|  CRD Fernando|             14.64706|           10|
|   8|    YA Abdulla|                 14.8|           11|
|   9|A Ashish Reddy|                 15.0|           20|
|  10|       S Gopal|             15.60526|           30|
+----+--------------+---------------------+-------------+
only showing top 10 rows



In [36]:
wta = spark.sql('select *,round((212-Rank)/212, 5) as Points,round(1.5*(212-Rank)/212, 5) as Weight \
                from wicket_taking_ability')
wta.registerTempTable('wicket_taking_ability')
wta.show(10)

+----+--------------+---------------------+-------------+-------+-------+
|Rank|        Bowler|Wicket_Taking_Ability|No_of_Matches| Points| Weight|
+----+--------------+---------------------+-------------+-------+-------+
|   1|       A Zampa|             11.84211|           11|0.99528|1.49293|
|   2| Sohail Tanvir|             12.04545|           11|0.99057|1.48585|
|   3|       K Ahmed|             12.68421|           10|0.98585|1.47877|
|   4|        N Rana|             13.42857|           12|0.98113|1.47170|
|   5|      K Rabada|                 14.0|           18|0.97642|1.46462|
|   6|      BJ Hodge|                 14.0|           20| 0.9717|1.45755|
|   7|  CRD Fernando|             14.64706|           10|0.96698|1.45047|
|   8|    YA Abdulla|                 14.8|           11|0.96226|1.44340|
|   9|A Ashish Reddy|                 15.0|           20|0.95755|1.43632|
|  10|       S Gopal|             15.60526|           30|0.95283|1.42925|
+----+--------------+-----------------

### Consistency 

In [37]:
# Consistency = Runs Conceded/Wickets Taken
cons = spark.sql('select t1.bowler as Bowler, round(t1.runs/t2.wickets, 5) as Consistency \
                 from (select sum(total_runs) as runs,bowler from analysis_db group by bowler) t1 \
                 inner join \
                 (Select bowler,count(*) as wickets from analysis_db where player_dismissed is not null \
                 and (dismissal_kind = \'bowled\' or  dismissal_kind = \'hit wicket\' \
                 or dismissal_kind = \'stumped\' or  dismissal_kind = \'lbw\' \
                 or dismissal_kind = \'caught and bowled\' or  dismissal_kind = \'caught\') \
                 group by bowler)t2 on t1.bowler = t2.bowler')
cons.registerTempTable('consistency')
cons.show(10)

+--------------+-----------+
|        Bowler|Consistency|
+--------------+-----------+
|         A Roy|       14.0|
| Kuldeep Yadav|   30.07692|
|    TM Dilshan|       73.6|
|       J Botha|      32.72|
|    KA Pollard|     31.875|
| LA Carseldine|        6.0|
|M Muralitharan|   27.57813|
|      DR Smith|   31.73077|
|Jaskaran Singh|       29.0|
|    A Flintoff|       53.0|
+--------------+-----------+
only showing top 10 rows



In [38]:
cons = spark.sql('select row_number() over (order by c.Consistency asc) as Rank, c.*,n.No_of_Matches \
                 from consistency c \
                 inner join \
                 no_of_matches_bowlers n on \
                 c.Bowler = n.Bowler where n.No_of_Matches>9')
cons.registerTempTable('consistency')
cons.show(10)

+----+--------------+-----------+-------------+
|Rank|        Bowler|Consistency|No_of_Matches|
+----+--------------+-----------+-------------+
|   1| Sohail Tanvir|       12.5|           11|
|   2|       A Zampa|   14.78947|           11|
|   3|        N Rana|   17.71429|           12|
|   4|  CRD Fernando|       18.0|           10|
|   5|      BJ Hodge|   18.23529|           20|
|   6|       K Ahmed|   18.47368|           10|
|   7|AD Mascarenhas|   19.21053|           13|
|   8|      K Rabada|   19.32258|           18|
|   9|  DE Bollinger|   19.35135|           27|
|  10|   MF Maharoof|    19.7037|           20|
+----+--------------+-----------+-------------+
only showing top 10 rows



In [39]:
cons = spark.sql('select *,round((212-Rank)/212, 5) as Points, round((212-Rank)*1.25/212, 5) as Weights from consistency')
cons.registerTempTable('consistency')
cons.show(10)

+----+--------------+-----------+-------------+-------+-------+
|Rank|        Bowler|Consistency|No_of_Matches| Points|Weights|
+----+--------------+-----------+-------------+-------+-------+
|   1| Sohail Tanvir|       12.5|           11|0.99528|1.24410|
|   2|       A Zampa|   14.78947|           11|0.99057|1.23821|
|   3|        N Rana|   17.71429|           12|0.98585|1.23231|
|   4|  CRD Fernando|       18.0|           10|0.98113|1.22642|
|   5|      BJ Hodge|   18.23529|           20|0.97642|1.22052|
|   6|       K Ahmed|   18.47368|           10| 0.9717|1.21462|
|   7|AD Mascarenhas|   19.21053|           13|0.96698|1.20873|
|   8|      K Rabada|   19.32258|           18|0.96226|1.20283|
|   9|  DE Bollinger|   19.35135|           27|0.95755|1.19693|
|  10|   MF Maharoof|    19.7037|           20|0.95283|1.19104|
+----+--------------+-----------+-------------+-------+-------+
only showing top 10 rows



### Crucial Wicket Taking Ability

In [40]:
# Crucial Wicket Taking Ability = Number of times Four or Five Wickets Taken/Number of Innings Played
cwta = spark.sql('select t2.bowler as Bowler, round(nvl(t1.no_of_4wickets/t2.innings,0), 5) as Crucial_Wicket_Taking_Ablity \
                 from (select bowler,count(*) as no_of_4wickets from (select * from \
                 (select match_id,bowler,count(*) as wickets from analysis_db where player_dismissed \
                 is not null \
                 and (dismissal_kind = \'bowled\' or  dismissal_kind = \'hit wicket\' \
                 or  dismissal_kind = \'stumped\' or  dismissal_kind = \'lbw\' \
                 or  dismissal_kind = \'caught and bowled\' or  dismissal_kind = \'caught\') \
                 group by bowler,match_id ) \
                 where wickets > 3) group by bowler)t1 \
                 right join \
                 (select bowler,count(match_id) as \
                 innings from (select distinct(match_id),bowler from analysis_db) \
                 group by bowler)t2 \
                 on t1.bowler = t2.bowler order by Crucial_Wicket_Taking_Ablity desc')
cwta.registerTempTable('crucial_wicket_taking_ablity')
cwta.show(10)

+---------------+----------------------------+
|         Bowler|Crucial_Wicket_Taking_Ablity|
+---------------+----------------------------+
|       A Joseph|                     0.33333|
|  Shoaib Akhtar|                     0.33333|
|  Sohail Tanvir|                     0.18182|
|     YA Abdulla|                     0.18182|
|       Umar Gul|                     0.16667|
|         AJ Tye|                     0.15385|
|        L Ngidi|                     0.14286|
|Karanveer Singh|                     0.11111|
|       S Curran|                     0.11111|
|       K Rabada|                     0.11111|
+---------------+----------------------------+
only showing top 10 rows



In [41]:
cwta = spark.sql('select rank() over (order by cw.Crucial_Wicket_Taking_Ablity desc) as Rank, cw.*,n.No_of_Matches \
                 from crucial_wicket_taking_ablity cw \
                 inner join no_of_matches_bowlers n on \
                 cw.Bowler = n.Bowler where n.No_of_Matches>9')
cwta.registerTempTable('crucial_wicket_taking_ablity')
cwta.show(10)

+----+-------------+----------------------------+-------------+
|Rank|       Bowler|Crucial_Wicket_Taking_Ablity|No_of_Matches|
+----+-------------+----------------------------+-------------+
|   1|Sohail Tanvir|                     0.18182|           11|
|   1|   YA Abdulla|                     0.18182|           11|
|   3|       AJ Tye|                     0.15385|           26|
|   4|     K Rabada|                     0.11111|           18|
|   5|     J Theron|                         0.1|           10|
|   5|  PC Valthaty|                         0.1|           10|
|   5| CRD Fernando|                         0.1|           10|
|   8|      A Zampa|                     0.09091|           11|
|   8|    CJ Jordan|                     0.09091|           11|
|  10|   A Chandila|                     0.08333|           12|
+----+-------------+----------------------------+-------------+
only showing top 10 rows



In [42]:
cwta = spark.sql('select *,round((212-Rank)/212, 5) as Points, round(1.25*(212-Rank)/212, 5) as Weights \
                 from crucial_wicket_taking_ablity')
cwta.registerTempTable('crucial_wicket_taking_ablity')
cwta.show(10)

+----+-------------+----------------------------+-------------+-------+-------+
|Rank|       Bowler|Crucial_Wicket_Taking_Ablity|No_of_Matches| Points|Weights|
+----+-------------+----------------------------+-------------+-------+-------+
|   1|Sohail Tanvir|                     0.18182|           11|0.99528|1.24410|
|   1|   YA Abdulla|                     0.18182|           11|0.99528|1.24410|
|   3|       AJ Tye|                     0.15385|           26|0.98585|1.23231|
|   4|     K Rabada|                     0.11111|           18|0.98113|1.22642|
|   5|     J Theron|                         0.1|           10|0.97642|1.22052|
|   5|  PC Valthaty|                         0.1|           10|0.97642|1.22052|
|   5| CRD Fernando|                         0.1|           10|0.97642|1.22052|
|   8|      A Zampa|                     0.09091|           11|0.96226|1.20283|
|   8|    CJ Jordan|                     0.09091|           11|0.96226|1.20283|
|  10|   A Chandila|                    

### Short Performance Index

In [43]:
# Short Performance Index = (Wickets Taken – 4* Number of Times Four Wickets Taken – 5* Number of Times Five Wickets Taken)/(Innings Played – Number of Times Four Wickets or Five Wickets Taken)
spi = spark.sql('select n.bowler as Bowler, round(nvl(t5.Short_Performance_Index,0), 5) as Short_Performance_Index \
                from (select t1.bowler,(t3.wickets - 4*t1.no_of_4wickets - 5*t2.no_of_4wickets)/ \
                (t4.innings - t1.no_of_4wickets - t2.no_of_4wickets) as Short_Performance_Index \
                from (select bowler,count(*) as no_of_4wickets \
                from (select * from (select match_id,bowler,count(*) as wickets from analysis_db where player_dismissed \
                is not null \
                and (dismissal_kind = \'bowled\' or  dismissal_kind = \'hit wicket\' \
                or  dismissal_kind = \'stumped\' or  dismissal_kind = \'lbw\' \
                or  dismissal_kind = \'caught and bowled\' or  dismissal_kind = \'caught\')\
                group by bowler, match_id ) \
                where wickets = 4) group by bowler) t1 \
                inner join \
                (select bowler,count(*) as no_of_4wickets from (select * from \
                (select match_id,bowler,count(*) as wickets from analysis_db where player_dismissed \
                is not null \
                and (dismissal_kind = \'bowled\' or  dismissal_kind = \'hit wicket\' \
                or dismissal_kind = \'stumped\' or  dismissal_kind = \'lbw\' \
                or dismissal_kind = \'caught and bowled\' or  dismissal_kind = \'caught\')\
                group by bowler,match_id ) \
                where wickets = 5) group by bowler) t2 \
                inner join \
                (select bowler,count(*) as wickets from analysis_db where player_dismissed is not null \
                and (dismissal_kind = \'bowled\' or  dismissal_kind = \'hit wicket\' \
                or  dismissal_kind = \'stumped\' or  dismissal_kind = \'lbw\' \
                or  dismissal_kind = \'caught and bowled\' or  dismissal_kind = \'caught\') \
                group by bowler) t3 \
                inner join \
                (select bowler,count(match_id) as \
                innings from (select distinct(match_id),bowler from analysis_db) group by bowler) t4 \
                on t1.bowler = t2.bowler and t1.bowler = t3.bowler and t1.bowler = t4.bowler) t5 \
                right join \
                no_of_matches_bowlers n on t5.Bowler = n.Bowler order by Short_Performance_Index desc')
spi.registerTempTable('short_performance_index')
spi.show(10)

+---------------+-----------------------+
|         Bowler|Short_Performance_Index|
+---------------+-----------------------+
|     SL Malinga|                1.22609|
|        B Kumar|                1.05263|
|       MM Patel|                1.01667|
|         AJ Tye|                    1.0|
|       A Mishra|                0.97203|
|      SP Narine|                0.91176|
|Harbhajan Singh|                0.90968|
|       L Balaji|                0.85507|
|    JP Faulkner|                0.82456|
|       A Kumble|                0.82051|
+---------------+-----------------------+
only showing top 10 rows



In [44]:
spi = spark.sql('select rank() over (order by sp.Short_Performance_Index desc) as Rank, sp.*, n.No_of_Matches \
                from short_performance_index sp \
                inner join \
                no_of_matches_bowlers n on \
                sp.Bowler = n.Bowler where n.No_of_Matches>9')
spi.registerTempTable('short_performance_index')
spi.show(10)

+----+---------------+-----------------------+-------------+
|Rank|         Bowler|Short_Performance_Index|No_of_Matches|
+----+---------------+-----------------------+-------------+
|   1|     SL Malinga|                1.22609|          122|
|   2|        B Kumar|                1.05263|          117|
|   3|       MM Patel|                1.01667|           63|
|   4|         AJ Tye|                    1.0|           26|
|   5|       A Mishra|                0.97203|          147|
|   6|      SP Narine|                0.91176|          109|
|   7|Harbhajan Singh|                0.90968|          157|
|   8|       L Balaji|                0.85507|           73|
|   9|    JP Faulkner|                0.82456|           60|
|  10|       A Kumble|                0.82051|           42|
+----+---------------+-----------------------+-------------+
only showing top 10 rows



In [45]:
spi = spark.sql('select *,round((212-Rank)/212, 5) as Points, round((212-Rank)*1.25/212, 5) as Weights \
                from short_performance_index')
spi.registerTempTable('short_performance_index')
spi.show(10)

+----+---------------+-----------------------+-------------+-------+-------+
|Rank|         Bowler|Short_Performance_Index|No_of_Matches| Points|Weights|
+----+---------------+-----------------------+-------------+-------+-------+
|   1|     SL Malinga|                1.22609|          122|0.99528|1.24410|
|   2|        B Kumar|                1.05263|          117|0.99057|1.23821|
|   3|       MM Patel|                1.01667|           63|0.98585|1.23231|
|   4|         AJ Tye|                    1.0|           26|0.98113|1.22642|
|   5|       A Mishra|                0.97203|          147|0.97642|1.22052|
|   6|      SP Narine|                0.91176|          109| 0.9717|1.21462|
|   7|Harbhajan Singh|                0.90968|          157|0.96698|1.20873|
|   8|       L Balaji|                0.85507|           73|0.96226|1.20283|
|   9|    JP Faulkner|                0.82456|           60|0.95755|1.19693|
|  10|       A Kumble|                0.82051|           42|0.95283|1.19104|

In [46]:
# Table Name for each Metric
# Economy: economy
# Wicket Taking Ability: wicket_taking_ability
# Consistency: consistency
# Crucial Wicket Taking Ablity: crucial_wicket_taking_ablity
# Short Performance Index: short_performance_index

## Total Bowling Weights

In [47]:
total_bowling_weight = spark.sql('select e.bowler as Bowler, round((e.Weight+wta.Weight+c.Weights+cwta.Weights+spi.Weights), 5) as Total_Bowling_Weights \
                                 from economy e \
                                 inner join wicket_taking_ability wta \
                                 on e.Bowler = wta.Bowler \
                                 inner join consistency c \
                                 on e.Bowler = c.Bowler \
                                 inner join crucial_wicket_taking_ablity cwta \
                                 on e.Bowler = cwta.Bowler \
                                 inner join short_performance_index spi \
                                 on e.Bowler = spi.Bowler \
                                 order by Total_Bowling_Weights desc')
total_bowling_weight.registerTempTable('total_bowling_weight')
total_bowling_weight.show(10)

+---------------+---------------------+
|         Bowler|Total_Bowling_Weights|
+---------------+---------------------+
|  Sohail Tanvir|              6.64623|
| AD Mascarenhas|              6.30544|
|   CRD Fernando|              6.30072|
|        A Zampa|              6.29482|
|     SL Malinga|              6.19339|
|   DE Bollinger|              6.09316|
|     A Chandila|              5.98704|
|       MA Starc|              5.96698|
|NM Coulter-Nile|              5.83844|
|      R Rampaul|              5.81841|
+---------------+---------------------+
only showing top 10 rows



In [48]:
# Dropping intermediate Tables
table_names = ['no_of_matches_bowlers', 'economy', 'wicket_taking_ability', 'consistency', 'crucial_wicket_taking_ablity', 'short_performance_index']
for table in table_names:
    cmd = 'drop table if exists {}'.format(table)
    drop = spark.sql(cmd)
check = spark.sql('show tables')
check.show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
|        |         analysis_db|       true|
|        |       deliveries_db|       true|
|        |          matches_db|       true|
|        |total_batting_weight|       true|
|        |total_bowling_weight|       true|
+--------+--------------------+-----------+



## Total Weights per Player

In [49]:
total_weight = spark.sql('select *, (coalesce(total_batting_weight, 0) + coalesce(total_bowling_weight, 0)) as Total_Weight\
                          from(select t1.batsman as Player, nvl(t1.Total_Batting_Weights,0) as Total_Batting_Weight, \
                          nvl(t2.Total_Bowling_Weights,0) as Total_Bowling_Weight\
                          from total_batting_weight t1 \
                          full outer join total_bowling_weight t2 \
                          on t1.batsman = t2.bowler) \
                          order by Total_Weight desc')
total_weight.registerTempTable('total_weight')
total_weight.show(10)

+-----------------+--------------------+--------------------+------------+
|           Player|Total_Batting_Weight|Total_Bowling_Weight|Total_Weight|
+-----------------+--------------------+--------------------+------------+
|        CH Morris|             6.42812|             4.98469|    11.41281|
|      Rashid Khan|             5.80000|             5.57784|    11.37784|
|      MF Maharoof|             5.58854|             5.77360|    11.36214|
|        SP Narine|             5.03021|             5.81840|    10.84861|
|        KK Cooper|             5.45834|             5.33256|    10.79090|
|    Mohammad Nabi|             5.71250|             5.04129|    10.75379|
|           AJ Tye|             5.15313|             5.41156|    10.56469|
|         M Morkel|             5.66458|             4.80189|    10.46647|
|Washington Sundar|             6.11771|             4.30543|    10.42314|
|   A Ashish Reddy|             5.72083|             4.66747|    10.38830|
+-----------------+------

In [50]:
# Dropping intermediate tables
table_names = ['total_batting_weight', 'total_bowling_weight']
for table in table_names:
    cmd = 'drop table if exists {}'.format(table)
    drop = spark.sql(cmd)
check = spark.sql('show tables')
check.show()

+--------+-------------+-----------+
|database|    tableName|isTemporary|
+--------+-------------+-----------+
|        |  analysis_db|       true|
|        |deliveries_db|       true|
|        |   matches_db|       true|
|        | total_weight|       true|
+--------+-------------+-----------+



In [52]:
# Saving the player weight data
total_weight.toPandas().to_csv(path + '/dataset/weights_data/player_weights.csv')

# Model Trainning

In [ ]:
# Importing Libraries
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics

In [ ]:
# Reading data
matches = pd.read_csv(path + '/dataset/clean_data/matches.csv')
deliveries = pd.read_csv(path + '/dataset/clean_data/deliveries.csv')

In [ ]:
encode = {'team1': {'Mumbai Indians':1,'Kolkata Knight Riders':2,'Royal Challengers Bangalore':3,
                             'Deccan Chargers':4,'Chennai Super Kings':5,'Rajasthan Royals':6,'Delhi Daredevils':7,
                             'Gujarat Lions':8,'Kings XI Punjab':9,'Sunrisers Hyderabad':10,'Rising Pune Supergiant':11,
                             'Kochi Tuskers Kerala':12,'Pune Warriors':13, 'Delhi Capitals':14},
                   'team2': {'Mumbai Indians':1,'Kolkata Knight Riders':2,'Royal Challengers Bangalore':3,
                             'Deccan Chargers':4,'Chennai Super Kings':5,'Rajasthan Royals':6,'Delhi Daredevils':7,
                             'Gujarat Lions':8,'Kings XI Punjab':9,'Sunrisers Hyderabad':10,'Rising Pune Supergiant':11,
                             'Kochi Tuskers Kerala':12,'Pune Warriors':13, 'Delhi Capitals':14},
                   'toss_winner': {'Mumbai Indians':1,'Kolkata Knight Riders':2,'Royal Challengers Bangalore':3,
                                   'Deccan Chargers':4,'Chennai Super Kings':5,'Rajasthan Royals':6,'Delhi Daredevils':7,
                                   'Gujarat Lions':8,'Kings XI Punjab':9,'Sunrisers Hyderabad':10,'Rising Pune Supergiant':11,
                                   'Kochi Tuskers Kerala':12,'Pune Warriors':13, 'Delhi Capitals':14},
                   'winner': {'Mumbai Indians':1,'Kolkata Knight Riders':2,'Royal Challengers Bangalore':3,'Deccan Chargers':4,
                              'Chennai Super Kings':5,'Rajasthan Royals':6,'Delhi Daredevils':7,'Gujarat Lions':8,
                              'Kings XI Punjab':9,'Sunrisers Hyderabad':10,'Rising Pune Supergiant':11,'Kochi Tuskers Kerala':12,
                              'Pune Warriors':13, 'Delhi Capitals':14, 'Draw':15}}
matches.replace(encode, inplace=True)

In [ ]:
# Checking the encoding result
matches.head(2)

In [ ]:
matches = matches[['team1','team2','city','toss_decision','toss_winner','venue','winner','season']]
df = pd.DataFrame(matches)

In [ ]:
var_mod = ['city','toss_decision','venue']
le = LabelEncoder()
for i in var_mod:
    df[i] = le.fit_transform(df[i])
df.dtypes

In [ ]:
# Apply RandomForest
model = RandomForestClassifier(n_estimators=100)
outcome = ['winner']
predictors = ['team1', 'team2', 'venue', 'toss_winner','city','toss_decision']
# classification_model(model, df,predictor_var,outcome_var)

model.fit(df[predictors]/, df[outcome].values.ravel())
predictions = model.predict(df[predictors])
accuracy = metrics.accuracy_score(predictions,df[outcome])
print('Accuracy : {0:.3}%'.format(100*accuracy))

kf = KFold(n_splits=7)
error = []
for train, test in kf.split(df):
    train_predictors = (df[predictors].iloc[train,:])
    train_target = df[outcome].iloc[train]
    model.fit(train_predictors, train_target.values.ravel())
    error.append(model.score(df[predictors].iloc[test,:], df[outcome].iloc[test]))

print('Cross-Validation Score : {0:.3%}'.format(np.mean(error)))
model.fit(df[predictors],df[outcome].values.ravel()) 

### Creating Player list based on teams player per season

In [53]:
players = spark.sql('select m.id, m.season, d.batting_team, d.bowling_team, d.batsman, d.non_striker, d.bowler from matches_db m \
                     full outer join deliveries_db d \
                     on m.id=d.match_id \
                     order by m.id')
players.toPandas().to_csv(path + '/dataset/players.csv')

In [54]:
players = pd.read_csv(path + '/dataset/players.csv')
player = set()
for i in range(len(players['id'])):
    player.add((players['season'][i],players['batting_team'][i],players['batsman'][i]))
    player.add((players['season'][i],players['batting_team'][i],players['non_striker'][i]))
    player.add((players['season'][i],players['bowling_team'][i],players['bowler'][i]))
    
pd.DataFrame(list(player), columns =['season', 'team', 'player']).to_csv(path + '/dataset/players.csv')

In [108]:
players = spark.read.csv(path + '/dataset/players.csv',inferSchema=True,header=True)
total_weight = spark.read.csv(path + '/dataset/weights_data/player_weights.csv',inferSchema=True,header=True)

In [109]:
players.registerTempTable('players')
players.show(10)

+---+------+--------------------+--------------+
|_c0|season|                team|        player|
+---+------+--------------------+--------------+
|  0|  2017|    Delhi Daredevils|    AD Mathews|
|  1|  2019|    Rajasthan Royals|    JC Buttler|
|  2|  2008| Chennai Super Kings|      S Vidyut|
|  3|  2013|    Rajasthan Royals|     STR Binny|
|  4|  2012|     Deccan Chargers|V Pratap Singh|
|  5|  2010|    Rajasthan Royals|      GC Smith|
|  6|  2012|       Pune Warriors|      M Manhas|
|  7|  2011|     Kings XI Punjab|      AM Nayar|
|  8|  2019|Kolkata Knight Ri...|    KD Karthik|
|  9|  2012|     Kings XI Punjab|  Bipul Sharma|
+---+------+--------------------+--------------+
only showing top 10 rows



In [110]:
total_weight.registerTempTable('player_weight')
total_weight.show(10)

+---+-----------------+--------------------+--------------------+------------+
|_c0|           Player|Total_Batting_Weight|Total_Bowling_Weight|Total_Weight|
+---+-----------------+--------------------+--------------------+------------+
|  0|        CH Morris|             6.42812|             4.98469|    11.41281|
|  1|      Rashid Khan|                 5.8|             5.57784|    11.37784|
|  2|      MF Maharoof|             5.58854|              5.7736|    11.36214|
|  3|        SP Narine|             5.03021|              5.8184|    10.84861|
|  4|        KK Cooper|             5.45834|             5.33256|     10.7909|
|  5|    Mohammad Nabi|              5.7125|             5.04129|    10.75379|
|  6|           AJ Tye|             5.15313|             5.41156|    10.56469|
|  7|         M Morkel|             5.66458|             4.80189|    10.46647|
|  8|Washington Sundar|             6.11771|             4.30543|    10.42314|
|  9|   A Ashish Reddy|             5.72083|        

In [111]:
team_weight = spark.sql('select * from players p \
                         left join player_weight pw \
                         on p.player = pw.player')
team_weight.registerTempTable('players')

In [118]:
team_weight = spark.sql('select team as Team, season as Season, sum(total_batting_weight) as Total_Batting_Weight,\
                         sum(total_bowling_weight) as Total_Bowling_Weight,\
                         sum(total_weight) as Team_Weight from players \
                         group by team, season \
                         order by season')
team_weight.show(10)
team_weight = team_weight.toPandas()

+--------------------+------+--------------------+--------------------+------------------+
|                Team|Season|Total_Batting_Weight|Total_Bowling_Weight|       Team_Weight|
+--------------------+------+--------------------+--------------------+------------------+
|      Mumbai Indians|  2008|            42.19584|  30.464669999999998| 72.66050999999999|
|Kolkata Knight Ri...|  2008|  35.896899999999995|  33.470600000000005|           69.3675|
|     Deccan Chargers|  2008|            34.21982|            29.40219|63.622009999999996|
|     Kings XI Punjab|  2008|            41.61356|  22.948169999999998|          64.56173|
| Chennai Super Kings|  2008|            42.07189|  18.717010000000002|           60.7889|
|    Rajasthan Royals|  2008|             29.6823|            34.32318|          64.00548|
|    Delhi Daredevils|  2008|            42.70313|            28.94229|          71.64542|
|Royal Challengers...|  2008|            41.54585|            32.39867|          73.94452|

In [119]:
# Converting to csv 
team_weight.to_csv(path + '/dataset/weights_data/team_weights.csv')

In [121]:
encode = {'Mumbai Indians':1,'Kolkata Knight Riders':2,'Royal Challengers Bangalore':3,
          'Deccan Chargers':4,'Chennai Super Kings':5,'Rajasthan Royals':6,'Delhi Daredevils':7,
          'Gujarat Lions':8,'Kings XI Punjab':9,'Sunrisers Hyderabad':10,'Rising Pune Supergiant':11,
          'Kochi Tuskers Kerala':12,'Pune Warriors':13, 'Delhi Capitals':14}
team_weight.replace(encode, inplace=True)
team_weight.head(10)